# Exploring OLS, Lasso and Random Forest in a regression task

In [4]:
# module imports
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

In [5]:
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/main/data/"

In [6]:
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [7]:
# examining the data
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [8]:
data.info()
# we can see that our data types are consitent and there are no null values
# so there's no need to preprocess anything

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [9]:
# let's separate the data 
y = data['quality']
data.drop(['quality'], axis=1, inplace=True)


In [10]:
# let's create a 30% holdout set for testing
X_train, X_test, y_train, y_test = train_test_split(
    data.values, y, test_size=0.3, random_state=17
)

In [11]:
# now we will scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Linear regression

In [12]:
# we will train a simple OLS model
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

### Question 1: What are mean squared errors of model predictions on train and holdout sets?

In [13]:
y_train_pred = linreg.predict(X_train_scaled)
y_test_pred = linreg.predict(X_test_scaled)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("Mean squared error (train): %.3f" % mse_train)
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (train): 0.558
Mean squared error (test): 0.584


### Question 2: Which feature this linear regression model treats as the most influential on wine quality?

In [14]:
# we will sort the features here by their influence on the target
linreg_coef = pd.DataFrame(linreg.coef_, index=[data.columns], columns=["coef"])
linreg_coef.sort_values(by="coef", key=(lambda x: -abs(x)), inplace=True)

In [15]:
linreg_coef

,coef
density,-0.665720
residual sugar,0.538164
volatile acidity,-0.192260
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127


## Lasso regression

In [16]:
# we will first train a LASSO model with a weak regularization
alpha = 0.01
lasso1 = Lasso(alpha=alpha, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

In [17]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=[data.columns], columns=['weight'])
lasso1_coef.sort_values(by='weight', key=(lambda x: -abs(x)))

,weight
alcohol,0.322425
residual sugar,0.256363
density,-0.235492
volatile acidity,-0.188479
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
chlorides,-0.002747
fixed acidity,-0.000000
citric acid,-0.000000


In [18]:
# we will train a LassoCV model to find the best alpha value
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(
    alphas=alphas, cv=5, n_jobs=-1, random_state=17
)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, n_jobs=-1, random_state=17)

In [19]:
lasso_cv.alpha_

0.0002833096101839324

### Question 3: Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?

In [20]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, index=data.columns, columns=['coef'])
lasso_cv_coef.sort_values(by='coef', key=(lambda x: -abs(x)))

,coef
density,-0.648161
residual sugar,0.526883
volatile acidity,-0.192049
pH,0.146549
alcohol,0.137115
fixed acidity,0.093295
sulphates,0.060939
free sulfur dioxide,0.042698
total sulfur dioxide,0.012969
chlorides,0.006933


### Question 4: What are mean squared errors of tuned LASSO predictions on train and holdout sets?

In [21]:
mse_lasso_train = mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))
mse_lasso_test = mean_squared_error(y_test, lasso_cv.predict(X_test_scaled))

In [22]:
print("Mean squared error (train): %.3f" % mse_lasso_train)
print("Mean squared error (test): %.3f" % mse_lasso_test)

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Random Forest

In [23]:
# we will train a random forest with default parameters
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

### Question 5: What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring=’neg_mean_squared_error’ and other arguments left with default values) and on holdout set?

In [24]:
y_train_pred = forest.predict(X_train_scaled)
y_test_pred = forest.predict(X_test_scaled)
mse_train = mean_squared_error(y_true=y_train, y_pred=y_train_pred)
mse_test = mean_squared_error(y_true=y_test, y_pred=y_test_pred)

In [25]:
cv_score_test = cross_val_score(
    estimator=forest, 
    X=X_test_scaled, 
    y=y_test,
    scoring="neg_mean_squared_error"
    )
mse_cv = np.mean(
    np.abs(cv_score_test)
)

In [26]:
print("Mean squared error (train): %.3f" % mse_train)
print("Mean squared error (cv): %.3f" % mse_cv)
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (train): 0.053
Mean squared error (cv): 0.451
Mean squared error (test): 0.372


### Question 6: What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring=’neg_mean_squared_error’ and other arguments left with default values) and on holdout set?

In [30]:
# we will tune our hyperparameters using gridsearch
forest_params = {
    'max_depth': list(range(10, 25)),
    'max_features': list(range(6,12))
    }

In [31]:
best_rf = GridSearchCV(
    RandomForestRegressor(n_jobs=-1, random_state=17),
    forest_params,   
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    cv=5,
    verbose=True
)

In [32]:
best_rf.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/home/oskorusha/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/oskorusha/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/oskorusha/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/oskorusha/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/oskorusha/.local/lib/pyt

KeyboardInterrupt: 

In [34]:
best_rf.best_params_, best_rf.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [41]:
best_rf = RandomForestRegressor(
    max_depth=21, max_features=6, n_jobs=-1, random_state=17
)
best_rf.fit(X_train_scaled, y_train)


RandomForestRegressor(max_depth=21, max_features=6, n_jobs=-1, random_state=17)

In [42]:
mse_cv = np.mean(
        np.abs(
            cross_val_score(
                best_rf,
                X_train_scaled,
                y_train,
                scoring="neg_mean_squared_error",
            )
        )
    )
mse_test = mean_squared_error(y_test, best_rf.predict(X_test_scaled))

In [43]:
print("Mean squared error (cv): %.3f" % mse_cv)
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (cv): 0.398
Mean squared error (test): 0.366


### Question 7: What is the most important feature, according to the Random Forest model?

In [48]:
rf_importance = pd.DataFrame(
    best_rf.feature_importances_,
    columns=['coef'],
    index=data.columns
)
rf_importance.sort_values(by='coef', key=(lambda x: -abs(x)))


,coef
alcohol,0.206056
volatile acidity,0.117578
free sulfur dioxide,0.111556
density,0.088549
pH,0.073659
total sulfur dioxide,0.073640
chlorides,0.073366
residual sugar,0.072072
citric acid,0.062601
fixed acidity,0.061813
